In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import gradio as gr
from collections import defaultdict

# Load pre-trained Faster R-CNN model from TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1")

# Function to perform object detection and analyze customer behavior
def analyze_retail(image):
    # Convert image to numpy array and resize
    image = np.array(image)
    resized_image = cv2.resize(image, (640, 640))

    # Convert to tensor and add batch dimension
    input_tensor = tf.convert_to_tensor(resized_image)
    input_tensor = tf.expand_dims(input_tensor, axis=0)

    # Perform object detection
    results = model(input_tensor)

    # Extract detection boxes, scores, and classes
    boxes = results["detection_boxes"].numpy()[0]  # Shape: (num_detections, 4)
    scores = results["detection_scores"].numpy()[0]  # Shape: (num_detections,)
    classes = results["detection_classes"].numpy()[0]  # Shape: (num_detections,)

    # Initialize counters for people and products
    people_count = 0
    product_count = 0
    product_interactions = defaultdict(int)

    # Draw bounding boxes on the image and count objects
    for i in range(len(boxes)):
        if scores[i] > 0.5:  # Only consider detections with confidence > 50%
            ymin, xmin, ymax, xmax = boxes[i]
            (left, right, top, bottom) = (int(xmin * 640), int(xmax * 640), int(ymin * 640), int(ymax * 640))

            # Class 1 is "person" in COCO dataset
            if int(classes[i]) == 1:
                cv2.rectangle(resized_image, (left, top), (right, bottom), (0, 255, 0), 2)  # Green box for people
                cv2.putText(resized_image, "Person", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                people_count += 1
            else:
                # Assume other classes are products
                cv2.rectangle(resized_image, (left, top), (right, bottom), (255, 0, 0), 2)  # Blue box for products
                cv2.putText(resized_image, f"Product {int(classes[i])}", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                product_count += 1
                product_interactions[int(classes[i])] += 1

    # Resize the annotated image back to the original size
    annotated_image = cv2.resize(resized_image, (image.shape[1], image.shape[0]))

    # Generate analytics
    analytics = {
        "people_count": people_count,
        "product_count": product_count,
        "product_interactions": dict(product_interactions),
    }

    return annotated_image, analytics

# Gradio interface for real-time retail analytics
interface = gr.Interface(
    fn=analyze_retail,
    inputs="image",
    outputs=["image", "json"],
    live=True,
    title="Retail Analytics Platform",
    description="Upload an image to analyze customer behavior and product interactions using a pre-trained Faster R-CNN model."
)
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15c3127d5639429869.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00
